In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import nltk
from nltk.stem import PorterStemmer as ps
from nltk.corpus import stopwords

In [ ]:
df_fake = pd.read_csv('/kaggle/input/fake-news-detection-datasets/News _dataset/Fake.csv')
df_fake.insert(column = 'classification', value= 'fake', loc =4)
df_true = pd.read_csv('/kaggle/input/fake-news-detection-datasets/News _dataset/True.csv')
df_true.insert(column = 'classification', value= 'true', loc = 4)
df = pd.concat([df_fake, df_true], ignore_index=True)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
df = df.drop(['date', 'title', 'subject'], axis = 1)

In [ ]:
df

In [ ]:
import re
stemmer = ps()
corpus = []

In [ ]:
for i in range(len(df['text'])):
  sent =  re.sub(r'[^\w\s]', '', df['text'][i]) #cleaning each sentence will remove all punctutation from it
  sent = sent.lower() #lower kr rhe hain sentence
  sent = sent.split() #list of words mil gye ab
  sent = [stemmer.stem(word) for word in sent if word not in set(stopwords.words('english'))]
  corpus.append(sent)

In [ ]:
corpus

In [ ]:
corpus_joined = [' '.join(token) for token in corpus]
print(corpus_joined)

In [ ]:
df.insert(2, "processed_text", corpus_joined, True)

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df["processed_text"], df["classification"], test_size = 0.2, random_state = 69)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
cx = TfidfVectorizer()
X_train_tfidf = cx.fit_transform(X_train).toarray()
X_test_tfidf = cx.transform(X_test).toarray()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train_tfidf, y_train)

In [ ]:
y_pred = decision_tree.predict(X_test_tfidf)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Decision Tree Accuracy: {accuracy:.2f}')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
random_forest = RandomForestClassifier()
random_forest.fit(X_train_tfidf, y_train)

In [ ]:
y_pred = random_forest.predict(X_test_tfidf)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Random Forest Accuracy: {accuracy:.2f}')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
clf = LogisticRegression(random_state=0)
clf.fit(X_train_tfidf, y_train)

In [ ]:
y_pred = clf.predict(X_test_tfidf)
acc = accuracy_score(y_test, y_pred)
print("Logistic Regression model accuracy (in %):", acc*100)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))